In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import scipy.integrate

# Get Data

In [ ]:
def getData(directory):
    #     end_time = 24 * 3600  # 24 hours
    for sdir_path, sdir in [
        (sdir.path, sdir.name)
        for sdir in sorted(
            os.scandir(directory), key=lambda x: int(x.name.split("_")[-1])
        )
        if sdir.is_dir()
    ]:
        print(sdir)
        for root, subdirs, files in os.walk(sdir_path):
            for file in files:
                dirs = root.split("/")
                if file == "0.trips.csv.gz" and "car" in dirs:
                    dists_car = pd.read_csv(os.path.join(root, file), sep=";")[
                        "traveled_distance"
                    ]
                if file == "0.vehicleDistanceStats_drt.csv" and "bimodal" in dirs:
                    dists_drt = pd.read_csv(os.path.join(root, file), sep=";")[
                        "drivenDistance_m"
                    ]
                if file == "trip_success.csv.gz" and "bimodal" in dirs:
                    rejections = pd.read_csv(os.path.join(root, file), sep=";")
                if file == "0.CummulativePtDistance.txt" and "bimodal" in dirs:
                    dist_train = pd.read_csv(os.path.join(root, file)).values[0, 0]
                if (
                    file == "0.drt_occupancy_time_profiles_drt.txt"
                    and "bimodal" in dirs
                ):
                    df_drt_occ = pd.read_csv(os.path.join(root, file), sep="\t")
                    df_drt_occ["time"] = df_drt_occ["time"].apply(timestmphm2sec)
                    drt_occ_non_empty, drt_deviation_non_empty = getAverageOcc(
                        df_drt_occ.drop(columns="time"), exclude_empty_vehicles=True
                    )
                    drt_occ, drt_deviation = getAverageOcc(
                        df_drt_occ.drop(columns="time")
                    )
                if file == "output_trips.csv.gz" and "bimodal" in dirs:
                    df_trips = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :,
                        [
                            "person",
                            "trav_time",
                            "wait_time",
                            "traveled_distance",
                            "modes",
                        ],
                    ]
                    df_trips["trav_time"] = df_trips["trav_time"].apply(timestmp2sec)
                    df_trips["wait_time"] = df_trips["wait_time"].apply(timestmp2sec)
                    df_trips.set_index("person", inplace=True)
                    df_trips = df_trips.groupby("person").agg(
                        {
                            "trav_time": np.sum,
                            "wait_time": np.sum,
                            "traveled_distance": np.sum,
                            "modes": combineModesSeriesStr,
                        }
                    )
                if file == "0.occupancyAnalysis.txt" and "bimodal" in dirs:
                    av_pt_occ, av_pt_occ_sq, n_pt = getPtOccupancies(
                        os.path.join(root, file), 600
                    )
                    t_av_pt_occ_av = getAverageTimeSeries(av_pt_occ)
                    sigma = np.sqrt(n_pt / (n_pt - 1)) * np.sqrt(
                        av_pt_occ_sq - av_pt_occ ** 2
                    )
                    t_av_pt_occ_sigma = getAverageTimeSeries(sigma)
                if file == "0.drt_trips_drt.csv" and "bimodal" in dirs:
                    drt_trip_dists = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :, "travelDistance_m"
                    ]
                if file == "0.drt_detours_drt.csv" and "bimodal" in dirs:
                    drt_detours = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :, "distanceDetour"
                    ]
                    #TODO fix this
                    drt_detours = drt_detours[drt_detours < 10]

        yield int(sdir.split("_")[-1]), dists_car, dists_drt, dist_train, rejections, (
            drt_occ_non_empty,
            drt_deviation_non_empty,
        ), (drt_occ, drt_deviation), df_trips["wait_time"].mean(), (
            t_av_pt_occ_av,
            t_av_pt_occ_sigma,
        ), df_trips[
            "modes"
        ].str.contains(
            "pt"
        ), df_trips[
            "traveled_distance"
        ], drt_trip_dists, drt_detours.mean()

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/WithUnimodal/varyingEllExt300drt/"
%matplotlib notebook

ls = np.empty(10)
carbon_indexs = np.empty(10)
cummulative_drt_car_bimodal = np.empty(10)
cummulative_train_bimodal = np.empty(10)
cummulative_car = np.empty(10)
av_drt_occs_non_empty = np.empty(10)
av_drt_occs = np.empty(10)
av_waits = np.empty(10)
serviced_fracts = np.empty(10)
av_pt_occs = np.empty(10)
bi_or_unimodal = np.empty((10,2))
drt_pt_person_km = np.empty((10,2))
av_detours = np.empty(10)

r0 = 3
# for x in getData(path):
#     fig,ax = plt.subplots()
#     x.plot(ax=ax)
#     continue
# plt.show()

for i, (
    ell,
    dists_car,
    dists_drt,
    dist_train,
    rejections,
    (drt_occ_non_empty, drt_occ_deviation_non_empty),
    (drt_occ, drt_occ_deviation),
    av_wait_time,
    (pt_occ, pt_occ_sigma),
    pt_in_trips,
    total_trip_dists,
    drt_trip_dists, av_detour
) in enumerate(getData(path)):
    
    # drt/pt_person_km can be get easier
    drt_pt_person_km[i,0] = drt_trip_dists.sum()
    drt_pt_person_km[i,1] = total_trip_dists.sum()-drt_pt_person_km[i,0]
    ls[i] = ell
    serviced_fracts[i] = len(rejections[rejections["tripSuccess"]]) / len(rejections)
    print("Servability: ", serviced_fracts[i])
    dists_car_for_rejected = dists_car.loc[
        rejections[~rejections["tripSuccess"]]["personId"].to_numpy()
    ]
    cummulative_drt_car_bimodal[i] = dists_drt.sum() + dists_car_for_rejected.sum()
    print("Distance by drt: ", dists_drt.sum())
    cummulative_train_bimodal[i] = dist_train
    cummulative_car[i] = dists_car.sum()
    carbon_index = (
        cummulative_drt_car_bimodal[i] + r0 * cummulative_train_bimodal[i]
    ) / cummulative_car[i]
    carbon_indexs[i] = carbon_index
    print(ell)
    print(carbon_index)
    av_drt_occs_non_empty[i] = drt_occ_non_empty
    av_drt_occs[i] = drt_occ
    av_waits[i] = av_wait_time
    av_pt_occs[i] = pt_occ
    vc_modes = pt_in_trips.value_counts()
    bi_or_unimodal[i,0] = vc_modes[False]
    bi_or_unimodal[i,1] = vc_modes[True]
    av_detours[i] = av_detour
    print("-----")

bi_or_unimodal = pd.DataFrame(bi_or_unimodal, index=ls, columns=["Unimodal","Bimodal"])
drt_pt_person_km = pd.DataFrame(drt_pt_person_km, index=ls, columns=["DRT","PT"])

---
## Theoretical prediction

In [ ]:
nu = 1 # per day
L = 1 #10km
E = 100000 #per 100km^2
R0 = 800/130
mu = 96 # per day
#nuELsquared = 1e5/(24*3600)
# delta = 1.5
# b = 2
zeta = 1
# U = 35
beta = 1/6*(np.sqrt(2)+np.log(1+np.sqrt(2)))
# f = 0.85

M = 10
ell_min = L/20
ell_max = L/2
ell_list = np.linspace(ell_min,ell_max,M)
#f = np.linspace(0,1,N) use f from simulations
f = serviced_fracts
Ci = np.zeros((M))
capacity = np.zeros((M))

def uniform_dist(x):
    if (x>L/2):
        return 0
    else:
        return 2/L
    
normalize = scipy.integrate.quad(uniform_dist,0,np.infty)[0]
assert np.isclose(1, normalize), "Should already be normalized"

def cummulative_prob(x):
    return scipy.integrate.quad(uniform_dist,0,x)[0]/normalize

kappa = np.zeros((M))

def size(x,dx):
    return uniform_dist(x)*np.arccos(dx/x)/(2*np.pi*normalize)

def tramsize(ell):
    scl = int(L/ell)
#    print(scl)
    sm = 0
    for i in np.linspace(0,L,100):#range(0,100):
        dx = ell*i
        sm = sm + 2*scipy.integrate.quad(size,np.maximum(i*ell,zeta*ell),np.infty,args=(dx))[0]
    sm = sm - (1-cummulative_prob(zeta*ell))/4
    sm = sm*nu*E*ell*ell # this is \mu*k ??
#     sm = 5/np.pi*E*nu*ell * (1-cummulative_prob(zeta*ell))
    
    return sm

def rpr(x):
    if x==0:
        return 0
    else:
        return x*uniform_dist(x)/normalize
    
def getCarbonIndex(ell, f, b, delta):
    den = nu*E*L*L*scipy.integrate.quad(rpr,0,np.infty)[0]

    for i in range(0,M):
        dbus = ((scipy.integrate.quad(rpr,0,zeta*ell[i]))[0] + 2*beta*ell[i]*(1-cummulative_prob(zeta*ell[i])))*nu*E*L*L
        #mu = tramsize(l[i])/100
        ctram = 4*mu*L*R0*L/ell[i]#+1)
#         print(ctram)
        Ci[i] = (((delta[i]/b[i])*f[i])*dbus + (1-f[i])*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L + ctram)/den
        cap = tramsize(ell[i])
        capacity[i] = cap*f[i]
        

getCarbonIndex(ell_list, f, av_drt_occs, av_detours)

In [ ]:
print(Ci, "\n", capacity, "\n", ell, "\n", f)

---
# Visualization

In [ ]:
%matplotlib notebook
save_figs = True

fig, ax = plt.subplots()
x = ls/10000
xlabel = r"$\ell/L$"
# Ci_old = Ci

for r0 in [R0]:#[1,2,3,4,5]:
    carbon_inds = (cummulative_drt_car_bimodal+r0*cummulative_train_bimodal)/cummulative_car
    ax.plot(x, carbon_inds, 'o-', label="Simulated (R0 ≈ {})".format(round(r0,1)))
#     print(np.argmin(carbon_inds))
    print("Min of C_I with r0=", round(r0,2), "at: ", x[np.argmin(carbon_inds)])

# ax.plot(ell_list, Ci_old, "o--", c='g', label="<b> counting standing vehicles: (R0 ≈ {})".format(round(r0,2)))
ax.plot(ell_list, Ci, "o--", c='r', label="Theory (R0 ≈ {})".format(round(r0,1)))
ax.set_ylabel(r"$C_I$")
print(carbon_inds/Ci)
print(ell_list[np.argmin(Ci)])
ax.set_xlabel(xlabel)
ax.legend()

ax.set_ylim((0,None))

if save_figs:
    fig.savefig("CarbonIdxVaryingl.png", dpi=200)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
bi_or_unimodal.set_index(np.round(x, decimals=2), inplace=True)
bi_or_unimodal.plot.bar(ax = ax, xlabel=xlabel, ylabel="frequency")

if save_figs:
    fig.savefig("BiOrUnimodalBarPlot.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
drt_pt_person_km.set_index(np.round(x, decimals=2), inplace=True)
drt_pt_person_km.plot.bar(ax = ax, xlabel=xlabel, ylabel="meter travelled")

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels)#, frameon=False, bbox_to_anchor=(1.05, 1))
display(drt_pt_person_km)

# save_figs = True
if save_figs:
    fig.savefig("PersonKmPerTravelMode.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, serviced_fracts, 'o-')
ax.set_ylabel("f")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("ServicedFraction.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_waits, 'o-')
ax.set_ylabel("Average waiting time (s)")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageWaitingTime.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_drt_occs_non_empty, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}^*\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageNonEmptyOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_drt_occs, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x, av_pt_occs, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{pt}\rangle$")
ax.set_xlabel(xlabel)

ax.plot(ell_list, capacity, "o--", c='r', label="Theory (R0 ≈ {})".format(round(r0,2)))

if save_figs:
    fig.savefig("AveragePtOccupancy.png", dpi=200)
plt.show()

In [ ]:

def tramsize(ell, mode='int'):
    scl = int(L/ell)
#    print(scl)
    sm = 0
    if mode=='int':
        for i in range(0,100):
            dx = ell*i
            sm = sm + 2*scipy.integrate.quad(size,np.maximum(i*ell,zeta*ell),np.infty,args=(dx))[0]
        sm = sm - (1-cummulative_prob(zeta*ell))/4
        sm = sm*nu*E*ell*ell # this is \mu*k ??
    elif mode=='av':
        sm = 5/np.pi*E*nu*ell * (1-cummulative_prob(zeta*ell))
    
    return sm

def rpr(x):
    if x==0:
        return 0
    else:
        return x*uniform_dist(x)/normalize
    
def getCarbonIndex(ell, f, b, delta):
    den = nu*E*L*L*scipy.integrate.quad(rpr,0,np.infty)[0]

    for i in range(0,M):
        dbus = ((scipy.integrate.quad(rpr,0,zeta*ell[i]))[0] + 2*beta*ell[i]*(1-cummulative_prob(zeta*ell[i])))*nu*E*L*L
        #mu = tramsize(l[i])/100
        cap = tramsize(ell[i])
        ctram = 2*mu*L*R0*L/ell[i]#+1)
        print(ctram)
        Ci[i] = (((delta[i]/b[i])*f[i])*dbus + (1-f[i])*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L + ctram)/den
        capacity[i] = cap*f[i]

def getTramsizes(ells):
    result = np.empty((len(ells),2))
    for i,ell in enumerate(ells):
        result[i,0] = tramsize(ell) 
        result[i,1] = tramsize(ell, mode='av')
    
    return result
        

getCarbonIndex(ell_list, f, av_drt_occs, av_detours)
tramsizes = getTramsizes(ell_list)

In [ ]:
plt.figure()
plt.scatter(ell_list, tramsizes[:,0], label='int')
plt.scatter(ell_list, tramsizes[:,1], label='av')
plt.legend()

plt.show()